# Implementación ALS

## LIBRERIAS

In [1]:
import pandas as pd
import numpy as np
import implicit
import scipy.sparse as sparse
import json

/home/martinrsts/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataframes


In [2]:
df_postulantes = pd.read_csv('postulantes_procesados.csv', index_col=0)
df_establecimientos = pd.read_csv('establecimientos_procesados.csv', index_col=0)
df_postulaciones_training = pd.read_csv('postulaciones_training.csv', index_col=0)
df_postulaciones_testing = pd.read_csv('postulaciones_testing.csv', index_col=0)

In [3]:
user_items_testing = {}

for row in df_postulaciones_testing.itertuples():
  if row[1] not in user_items_testing:
      user_items_testing[row[1]] = []
  user_items_testing[row[1]].append(row[2])

In [4]:
# Definicion de métricas (No editar)
# Obtenido de https://gist.github.com/bwhite/3726239

def precision_at_k_perso(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k_perso(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

def dcg_at_k(r, k):
    r = np.asarray(r, dtype=float)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

def recall_at_k(r):
    recall = np.mean(r)
    return recall

In [5]:
user_items_training = {}
itemset_training = set()

i = 0
for row in df_postulaciones_training.itertuples():
  if row[1] not in user_items_training:
      user_items_training[row[1]] = []

  user_items_training[row[1]].append(row[2])
  itemset_training.add(row[2])

itemset_training = np.sort(list(itemset_training))


In [6]:
sparse_matrix = np.zeros((len(user_items_training), len(itemset_training)))
userid_to_i = {}

for i, items in enumerate(user_items_training.values()):
    sparse_matrix[i] = np.isin(itemset_training, items, assume_unique=True).astype(int)


matrix = sparse.csr_matrix(sparse_matrix.T)

user_ids = {key: i for i, key in enumerate(user_items_training.keys())}
user_item_matrix = matrix.T.tocsr()

In [7]:
def evaluate_model(model, n):
  mean_map = 0.
  mean_ndcg = 0.
  mean_recall = 0.
  for u, c in zip(user_items_testing.keys(), user_items_testing.keys()):
    c = user_ids[c]
    rec = model.recommend(c, user_item_matrix[c], n)[0]
    rel_vector = [np.isin(itemset_training[rec], user_items_testing[u], assume_unique=True).astype(int)]
    vector_for_recall = [np.isin(user_items_testing[u], itemset_training[rec], assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, n)
    mean_recall += recall_at_k(vector_for_recall)

  mean_map /= len(user_items_testing)
  mean_ndcg /= len(user_items_testing)
  mean_recall /= len(user_items_testing)

  return mean_map, mean_ndcg, mean_recall

In [8]:
# Definimos y entrenamos el modelo con optimización ALS
model_als = implicit.als.AlternatingLeastSquares(factors=100, iterations=100, use_gpu=False)
model_als.fit(user_item_matrix)

/home/martinrsts/.local/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 6 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 100/100 [00:56<00:00,  1.78it/s]


In [9]:
maprec, ndcg, recall = evaluate_model(model_als, n=20)
print('map: {}\nndcg: {}\nrecall: {}'.format(maprec, ndcg, recall))

map: 0.2534925287956621
ndcg: 0.6466527347409382
recall: 0.5733189839073679


In [10]:
recomendaciones = {}

for u, c in zip(user_items_testing.keys(), user_items_testing.keys()):
    c = user_ids[c]
    rec = model_als.recommend(c, user_item_matrix[c], 20)[0]
    recomendaciones[u] = [int(i) for i in itemset_training[rec]]

In [ ]:
with open('../Visualizacion/data/recomendaciones_als.json', 'w') as output:
    json.dump(recomendaciones, output)